# Settings

In [ ]:
import numpy as np
import torch
import os
from module_space import *

SEED = 1234
np.random.seed(SEED)

if not os.path.exists('regrets'):
    os.mkdir('regrets')

#if not os.path.exists('plots'):
#    os.mkdir('plots')
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set()

### Bandit settings

In [60]:
T = 1000
n_arms = 12 # N
n_features_default = 20 # d
n_assortment = 4 # K
n_samples = 30 # M, number of samples per each round and arm, for TS
noise_std = 0.01 # noise of reward: xi = noise_std*N(0,1)

n_sim = 10 # number of simulations

def F(x): # round_reward_function
    if len(np.shape(x)) == 1: # if x is a vector
        return np.sum(x)
    else: # if x is a matrix
        return np.sum(x, axis=1)                

### Parameter settings

In [61]:
reg_factor = 1.0 # lambda
delta = 0.1 # delta
exploration_variance = 0.1 # nu, for TS
confidence_scaling_factor = 1.0 # gamma, for UCB

### Neural network settings

In [62]:
hidden_size_default = 60 # m
epochs = 100 # repeat training for each period
train_every = 10 # training period
training_window = 100
learning_rate = 0.01
p = 0.2 # dropout

use_cuda = False

#if torch.cuda.is_available():
#    use_cuda = True
#else:
#    use_cuda = False

# Experiment descirption
### Hidden functions
- Linear: $h_{1}(x_{t,i}) = a^{\top}x_{t,i}$
- Quadratic: $h_{2}(x_{t,i}) = (a^{\top}x_{t,i})^{2}$
- Non-linear: $h_{3}(x_{t,i}) = \cos(\pi a^{\top}x_{t,i})$
- where $a \sim N(0,1)/\|a\|_{2}$

### For each hidden function, compare the following algorithms
- CombLinUCB
- CombLinTS
- CN-UCB
- CN-TS(1): single reward sample
- CN-TS(30): optimistic sampling, sample size = 30 (default sample size is 1)

### Ablation study of feature dimension *d* and neural network width *m*
- Default value: $d = 20, m = 60$
- $d = \{20, 100\}$  for all algorithms
- $d = \{20, 40, 60, 80, 100\}$ for CN-UCB, CN-TS(1), and CN-TS(30)
- $m = \{20, 40, 60, 80, 100\}$ for CN-UCB, CN-TS(1), and CN-TS(30)

# Experiment settings

### Hidden function, bandit, learning algorithm and regret settings

In [63]:
h1 = "h1"
h2 = "h2"
h3 = "h3"

In [64]:
def experiment(lin_neural, ucb_ts, h_str, n_features=20, hidden_size=100, n_samples=1, save = ''):
    """ kind explanation
    """
    
    ## Hidden function    
    a = np.random.randn(n_features)
    a /= np.linalg.norm(a, ord=2)
    
    if h_str == "h1":
        h = lambda x: np.dot(a, x)
    elif h_str == "h2":
        h = lambda x: np.dot(a, x)**2
    elif h_str == "h3":
        h = lambda x: np.cos(np.pi*np.dot(x, a))
    
    ## Bandit
    bandit = ContextualBandit(T,
                              n_arms,
                              n_features, 
                              h,
                              noise_std=noise_std,
                              n_assortment=n_assortment,
                              n_samples=n_samples,
                              round_reward_function=F
                             )
    
    ## Learning algorithm and regret
    regrets = np.empty((n_sim, T))
    
    for i in range(n_sim):
        bandit.reset_rewards()
        
        if lin_neural == Neural:
            model = lin_neural(ucb_ts,
                               bandit,
                               hidden_size=hidden_size,
                               reg_factor=reg_factor,
                               delta=delta,
                               confidence_scaling_factor=confidence_scaling_factor,
                               exploration_variance=exploration_variance,
                               p=p,
                               training_window=training_window,
                               learning_rate=learning_rate,
                               epochs=epochs,train_every=train_every,
                               use_cuda=use_cuda
                              )
            
            model.set_init_param(model.model.parameters()) # keep initial parameters for regularization
            
        if lin_neural == Lin:
            model = lin_neural(ucb_ts,
                               bandit,
                               reg_factor=reg_factor,
                               delta=delta,
                               confidence_scaling_factor=confidence_scaling_factor,
                               exploration_variance=exploration_variance
                              )
        
        model.run()
        regrets[i] = np.cumsum(model.regrets)
    
    if save: # save regrets
        np.save('regrets/' + save, regrets)    

# Experiment A: Algorithm vs Regret

### CN-UCB

In [68]:
lin_neural = Neural
ucb_ts = "UCB"

##### Experiment A1

In [69]:
## d = 20, m = 60
n_features = n_features_default
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNUCB_20_60')

In [71]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNUCB_20_60')

100%|██████████| 1000/1000 [01:02<00:00, 16.06it/s, total regret=21.8, % optimal set of arms=30.80%]


In [72]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNUCB_20_60')

100%|██████████| 1000/1000 [01:00<00:00, 16.44it/s, total regret=198, % optimal set of arms=0.40%]


##### Experiment A2

In [ ]:
## d = 60, m = 60
n_features = 100
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNUCB_100_60')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNUCB_100_60')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNUCB_100_60')

### CN-TS

In [73]:
lin_neural = Neural
ucb_ts = "TS"

##### Experiment A3

In [74]:
## d = 20, m = 60
n_features = n_features_default
hidden_size = hidden_size_default

In [67]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNTS_20_60')

100%|██████████| 1000/1000 [01:00<00:00, 16.54it/s, total regret=14, % optimal set of arms=64.30%] 


In [75]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNTS_20_60')

100%|██████████| 1000/1000 [01:09<00:00, 14.36it/s, total regret=98.1, % optimal set of arms=17.80%]


In [76]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNTS_20_60')

100%|██████████| 1000/1000 [01:01<00:00, 16.31it/s, total regret=208, % optimal set of arms=0.40%]


##### Experiment A4

In [ ]:
## d = 60, m = 60
n_features = 100
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNTS_100_60')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNTS_100_60')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNTS_100_60')

### CN-TS-Opt

In [77]:
lin_neural = Neural
ucb_ts = "TS"

##### Experiment A5

In [78]:
## d = 20, m = 60
n_features = n_features_default
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples, save='reg_h1_CNTSOpt_20_60')

In [79]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples, save='reg_h2_CNTSOpt_20_60')

100%|██████████| 1000/1000 [00:57<00:00, 17.27it/s, total regret=21.8, % optimal set of arms=32.10%]


In [80]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples, save='reg_h3_CNTSOpt_20_60')

100%|██████████| 1000/1000 [00:57<00:00, 17.41it/s, total regret=246, % optimal set of arms=0.50%]


##### Experiment A6

In [ ]:
## d = 100, m = 60
n_features = 100
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples, save='reg_h1_CNTSOpt_100_60')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples, save='reg_h2_CNTSOpt_100_60')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples, save='reg_h3_CNTSOpt_100_60')

### CombLinUCB

In [ ]:
lin_neural = Lin
ucb_ts = "UCB"

##### Experiment A7

In [ ]:
## d = 20
n_features = n_features_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, save='reg_h1_CombLinUCB_20')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, save='reg_h2_CombLinUCB_20')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, save='reg_h3_CombLinUCB_20')

##### Experiment A8

In [ ]:
## d = 100
n_features = 100

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, save='reg_h1_CombLinUCB_100')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, save='reg_h2_CombLinUCB_100')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, save='reg_h3_CombLinUCB_100')

### CombLinTS

In [ ]:
lin_neural = Lin
ucb_ts = "TS"

##### Experiment A9

In [ ]:
## d = 20
n_features = n_features_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, save='reg_h1_CombLinTS_20')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, save='reg_h2_CombLinTS_20')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, save='reg_h3_CombLinTS_20')

##### Experiment Aa

In [ ]:
## d = 100
n_features = 100

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, save='reg_h1_CombLinTS_100')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, save='reg_h2_CombLinTS_100')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, save='reg_h3_CombLinTS_100')

# Experiment B: Ablation study

### CN-UCB

In [ ]:
lin_neural = Neural
ucb_ts = "UCB"

##### Experiment B01

In [ ]:
## d = 40, m = 60
n_features = 40
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNUCB_40_60')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNUCB_40_60')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNUCB_40_60')

##### Experiment B02

In [ ]:
## d = 60, m = 60
n_features = 60
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNUCB_60_60')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNUCB_60_60')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNUCB_60_60')

##### Experiment B03

In [ ]:
## d= 80, m = 60
n_features = 80
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNUCB_80_60')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNUCB_80_60')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNUCB_80_60')

##### Experiment B04

In [ ]:
## d = 20, m = 20
n_features = n_features_default
hidden_size = 20

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNUCB_20_20')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNUCB_20_20')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNUCB_20_20')

##### Experiment B05

In [ ]:
## d = 20, m = 40
n_features = n_features_default
hidden_size = 40

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNUCB_20_40')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNUCB_20_40')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNUCB_20_40')

##### Experiment B06

In [ ]:
## d = 20, m = 80
n_features = n_features_default
hidden_size = 80

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNUCB_20_80')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNUCB_20_80')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNUCB_20_80')

##### Experiment B07

In [ ]:
## d = 20, m = 100
n_features = n_features_default
hidden_size = 100

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNUCB_20_100')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNUCB_20_100')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNUCB_20_100')

### CN-TS

In [ ]:
lin_neural = Neural
ucb_ts = "TS"

##### Experiment B11

In [ ]:
## d = 40, m = 60
n_features = 40
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNTS_40_60')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNTS_40_60')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNTS_40_60')

##### Experiment B12

In [ ]:
## d = 60, m = 60
n_features = 60
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNTS_60_60')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNTS_60_60')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNTS_60_60')

##### Experiment B13

In [ ]:
## d= 80, m = 60
n_features = 80
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNTS_80_60')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNTS_80_60')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNTS_80_60')

##### Experiment B14

In [ ]:
## d = 20, m = 20
n_features = n_features_default
hidden_size = 20

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNTS_20_20')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNTS_20_20')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNTS_20_20')

##### Experiment B15

In [ ]:
## d = 20, m = 40
n_features = n_features_default
hidden_size = 40

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNTS_20_40')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNTS_20_40')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNTS_20_40')

##### Experiment B16

In [ ]:
## d = 20, m = 80
n_features = n_features_default
hidden_size = 80

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNTS_20_80')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNTS_20_80')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNTS_20_80')

##### Experiment B17

In [ ]:
## d = 20, m = 100
n_features = n_features_default
hidden_size = 100

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, save='reg_h1_CNTS_20_100')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, save='reg_h2_CNTS_20_100')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, save='reg_h3_CNTS_20_100')

### CN-TS-Opt

In [ ]:
lin_neural = Neural
ucb_ts = "TS"

##### Experiment B21

In [ ]:
## d = 40, m = 60
n_features = 40
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples, save='reg_h1_CNTSOpt_40_60')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples, save='reg_h2_CNTSOpt_40_60')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples, save='reg_h3_CNTSOpt_40_60')

##### Experiment B22

In [ ]:
## d = 60, m = 60
n_features = 60
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples, save='reg_h1_CNTSOpt_60_60')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples, save='reg_h2_CNTSOpt_60_60')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples, save='reg_h3_CNTSOpt_60_60')

##### Experiment B23

In [ ]:
## d= 80, m = 60
n_features = 80
hidden_size = hidden_size_default

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples, save='reg_h1_CNTSOpt_80_60')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples, save='reg_h2_CNTSOpt_80_60')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples, save='reg_h3_CNTSOpt_80_60')

##### Experiment B24

In [ ]:
## d = 20, m = 20
n_features = n_features_default
hidden_size = 20

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples, save='reg_h1_CNTSOpt_20_20')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples, save='reg_h2_CNTSOpt_20_20')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples, save='reg_h3_CNTSOpt_20_20')

##### Experiment B25

In [ ]:
## d = 20, m = 40
n_features = n_features_default
hidden_size = 40

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples, save='reg_h1_CNTSOpt_20_40')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples, save='reg_h2_CNTSOpt_20_40')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples, save='reg_h3_CNTSOpt_20_40')

##### Experiment B26

In [ ]:
## d = 20, m = 80
n_features = n_features_default
hidden_size = 80

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples, save='reg_h1_CNTSOpt_20_80')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples, save='reg_h2_CNTSOpt_20_80')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples, save='reg_h3_CNTSOpt_20_80')

##### Experiment B27

In [ ]:
## d = 20, m = 100
n_features = n_features_default
hidden_size = 100

In [ ]:
## regrets for h1, h2, h3
experiment(lin_neural, ucb_ts, h1, n_features, hidden_size, n_samples=n_samples, save='reg_h1_CNTSOpt_20_100')

In [ ]:
experiment(lin_neural, ucb_ts, h2, n_features, hidden_size, n_samples=n_samples, save='reg_h2_CNTSOpt_20_100')

In [ ]:
experiment(lin_neural, ucb_ts, h3, n_features, hidden_size, n_samples=n_samples, save='reg_h3_CNTSOpt_20_100')